# Libs

In [ ]:
from xml.dom.minidom import parse
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Read File

In [ ]:
def read_prog(rootNode):

    # generate record(list)
    rec = []
    for progs in rootNode.getElementsByTagName('progs'):
        for prog_idx, program in enumerate(progs.getElementsByTagName('prog')):
            cycle_len = program.getAttribute("cycletime")
            for sig_group in program.getElementsByTagName('sg'):
                sig_group_id = sig_group.getAttribute("sg_id")
                gr = sig_group.getElementsByTagName('cmds')[0].getElementsByTagName('cmd')
                for color in gr:
                    c = color.getAttribute('display')
                    t = color.getAttribute('begin')
                    rec.append([prog_idx + 1, cycle_len, sig_group_id, c, t])
    
    # turn into pd.DataFrame format
    prog = pd.DataFrame(rec, columns=['progId', 'cycleLen',
                                      'sigGroup', 'display', 'beginTime'])
    
    # post-process
    for col in prog.columns:
        prog[col] = pd.to_numeric(prog[col])
    
    prog['cycleLen'] /= 1000
    prog['beginTime'] /= 1000
    prog = prog.astype(int)
    prog['display'] = prog['display'].replace({3:'green', 1:'red'})
    
    return prog

In [ ]:
def viz_prog(prog, progId=1):
    
    fig = plt.figure(figsize=(7,5))
    cycle = prog[prog.progId == progId]['cycleLen'].unique()[0]
    for idx, sg_id in enumerate(prog['sigGroup'].unique()):
        sig_group = prog[prog.sigGroup == sg_id]
        g_duration = sig_group[sig_group.display == 'green']['beginTime'].iloc[0]
        r_duration = sig_group[sig_group.display == 'red']['beginTime'].iloc[0]

        if r_duration > g_duration:
            plt.plot(range(0, g_duration), np.tile(idx, g_duration), 'r', linewidth = 5)
            plt.plot(range(r_duration, cycle), np.tile(idx, cycle - r_duration), 'r', linewidth = 5)
            plt.plot(range(g_duration, r_duration), np.tile(idx, r_duration - g_duration), 'g', linewidth = 8)
        elif r_duration == 0:
            plt.plot(range(0, g_duration), np.tile(idx, g_duration), 'r', linewidth = 5)
            plt.plot(range(g_duration, cycle), np.tile(idx, cycle - g_duration), 'g', linewidth = 8)

    plt.xlabel('Cycle Time (s)')
    plt.ylabel('Signal Group (#)')
    plt.xlim([0,cycle])
    plt.ylim([-0.5,idx + 0.5])
    plt.yticks(np.arange(8), np.flip(np.arange(1,9)))
    
    return fig

In [ ]:
# read raw file
filepath = '../data/晋陵北路-河海路.sig'
domTree = parse(filepath)
rootNode = domTree.documentElement

# read programs from raw file
progs = read_prog(rootNode)

# visualize a program
fig = viz_prog(progs, progId=2)